In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import date
import seaborn as sns
import random

import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.mixture import GaussianMixture


from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox

import scipy.stats as stats
from scipy.stats import probplot, laplace, norm, t


import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess

import pymc as pm
import pytensor.tensor as pt
import arviz as az

import tensorflow as tf
from tensorflow import keras


#from tensorflow.keras.utils import plot_model


######################################
#from pmdarima import auto_arima
#from diptest import diptest

In [ ]:
class StockPredictor:
    def __init__(self, ticker, start, end, column, nfuture = 252):

        self.ticker = ticker
        self.start = start
        self.end = end
        self.model_type = None
        self.df=None
        self.scaler = MinMaxScaler(feature_range=(0, 1))
        self.nfuture = nfuture
        self.column=column
        self.rfr = 0.07
        self.risk_neutral = False


    def data_initiation(self):
        self.df = StockData(self.ticker, self.start, self.end, self.column).fetch_data()
        self.compute_log_return()
        self.df.head(5)
        return self.df

    def compute_log_return(self):
        """
        Compute log returns and store in self.df['LogReturn'].
        log_return = ln(P_t / P_{t-1})
        """
        self.df['LogReturn'] = np.log(self.df['Close'] / self.df['Close'].shift(1))

    def run_analytical(self, df, alpha, window, horizon, var_type, dist_type):
        var_bt = AnalyticalVaRBacktester(df, alpha, window, horizon, var_type, dist_type)
        var_bt.compute_var()
        var_bt.compute_future_log_return()
        var_bt.backtest()
        var_bt.plot_var()

    def run_empirical(self, df, alpha, window, horizon, var_type):
        var_bt = HistoricalVaRBacktester(df, alpha, window, horizon, var_type)
        var_bt.compute_var()
        var_bt.compute_future_log_return()
        var_bt.backtest()
        var_bt.plot_var()

    def run_monte_carlo(self, df, alpha, window, horizon, var_type, nsims_mc):
        var_bt = MonteCarloVaRBacktester(df, alpha, window, horizon, var_type, nsims_mc)
        var_bt.compute_var()
        var_bt.compute_future_log_return()
        var_bt.backtest()
        var_bt.plot_var()



